# Stabel Diffusion Model, using PyTorch

References:

1. [PyTorch docs](https://pytorch.org/TensorRT/tutorials/_rendered_examples/dynamo/torch_compile_stable_diffusion.html)

# Imports and consts

In [2]:
import torch
import torch_tensorrt
from diffusers import DiffusionPipeline

ModuleNotFoundError: No module named 'torch_tensorrt'

In [ ]:

model_id = "CompVis/stable-diffusion-v1-4"
device = "cpu" #cuda:0

# Model Definition

In [ ]:
# Instantiate Stable Diffusion Pipeline with FP16 weights
pipe = DiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
)
pipe = pipe.to(device)

backend = "torch_tensorrt"

# Optimize the UNet portion with Torch-TensorRT
pipe.unet = torch.compile(
    pipe.unet,
    backend=backend,
    options={
        "truncate_long_and_double": True,
        "enabled_precisions": {torch.float32, torch.float16},
    },
    dynamic=False,
)

# Inference

In [ ]:
prompt = "a majestic castle in the clouds"
image = pipe(prompt).images[0]

image.save("images/majestic_castle.png")
image.show()